In [1]:
import pandas as pd
import itertools
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook as tqdm

In [2]:
data = pd.read_csv('Data.csv')

In [15]:
#User Input
lower_bound_sma20 = -5,-3,2
upper_bound_sma20 = -2,-1,1
momentum_threshold = -50,50,25
up_from_day = 5,10,4
up_from_open = 10,14,2
lower_bound_rsi = 50,70,15
upper_bound_rsi = 60,80,15
average_volume_ratio = 0.5,2.5,1.5
market_cap_threshold = 500,5000,2000
min_event = 30

data['mrkt cap'] = data['mrkt cap'].fillna(0) #Fill Blank space in market cap with 0 

switch = list(itertools.product(['ON', 'OFF'], ['ON', 'OFF'], ['ON', 'OFF'], ['ON', 'OFF'], ['ON', 'OFF'], ['ON', 'OFF'],\
                       ['ON', 'OFF'], ['ON', 'OFF'], ['ON', 'OFF'])) #Get all the permutation of 'ON' and 'OFF'

result = pd.DataFrame() #Create an empty DataFrame to store result
col_name = [] #Create an empty list

event = ['win', 'upwin', 'Loss', 'update loss'] #Get all the events in a list

for j in tqdm(switch): #Iterate
    switch_lis = list(j) #Get the current permutation in a list

    #Assign default value to column will change if the condition is met
    data['sma20 > sma50'] = 1 
    data['MACD'] = 1
    try:
        data.loc[(data['20SMA'] > data['50SMA'] ) , 'sma20 > sma50'] = 0
    except:
        pass
    try:
        data.loc[(data['Macd'] == 'Pass') , 'MACD'] = 0
    except:
        pass

    off_index = [i for i, x in enumerate(switch_lis) if x == "OFF"] #Get the indices of OFF in a list
    off_column = off_index.copy() #Get the copy of above list. This will be used later
    #If the indices contains 0 or 8. Remove it. Since 'sma20 > sma50 check' and 'MACD pass/fail check' doesnt have value
    if 0 in off_index:
        off_index.remove(0)    
    if 8 in off_index:
        off_index.remove(8)

    del_col = [x+1 if x >= 6 else x for x in off_index] #Since rsi lower and upper bound are sharing a switch add 1 to variable coming after

    #Since 2 variables are sharing switch. Add another index if the other exist aligns variables indices with switch's (OFF only)
    for l in del_col:
        if (l == 1):
            del_col.append(0)
        if (l == 5):
            del_col.append(6)

    del_col.sort() #Sort the indices of off (THIS IS IMPORTANT)

    threshold_list = [lower_bound_sma20, upper_bound_sma20, momentum_threshold, up_from_day, up_from_open, lower_bound_rsi, \
                     upper_bound_rsi, average_volume_ratio, market_cap_threshold] #Get the low high step of variables in a list
    
    #Remove variables whose switch is off to reduce no of iterations
    for ele in sorted(del_col, reverse = True): 
        del threshold_list[ele]

    #Expand low, high with step of all variables. Eg: low 500 high 5000 step 2000 will give 500, 2500, 4500
    iteration = []
    for s in threshold_list:
        lis = list(s)
        start = [lis[0]]
        low = lis[0]
        while low <= s[1]:
            low += s[2]
            start.append(low)
        start.pop()
        iteration.append(start)
    
    iterate = list(itertools.product(*iteration)) #Create all possible iterations
    
    #Add 'OFF' in the specified index for all the iterations created in line 66 line 86 to 92
    scenarios = []
    for j in iterate:  
        lis = list(j)
        for k in del_col:
            lis.insert(k, 'OFF')
        scenarios.append(lis)
        
    for lis in scenarios: #Variable Loop
        #Create columns with default values. Will change when conditions are met
        data['sma20 bound check'] = 1
        data['momentum'] = 1
        data['8% Daily'] = 1
        data['2% UP'] = 1
        data['RSI (60-72)'] = 1
        data['Volume Ration'] = 1
        data['market cap'] = 0

        #Change the values created in column created above based on conditions. Will ignore if it is 'OFF'
        try:
            data.loc[(data['20SMA %'] < lis[1]) & (data['20SMA %'] > lis[0]), 'sma20 bound check'] = 0    
        except:
            pass
        try:
            data.loc[(data['Mom'] > lis[2]) , 'momentum'] = 0
        except:
            pass
        try:
            data.loc[(data['%Up'] > lis[3]) , '8% Daily'] = 0
        except:
            pass
        try:
            data.loc[(data['% Open'] > lis[4]) , '2% UP'] = 0
        except:
            pass
        try:
            data.loc[(data['14minRSI'] < lis[6]) & (data['14minRSI'] > lis[5]), 'RSI (60-72)'] = 0
        except:
            pass
        try:
            data.loc[(data['Volume Ratio'] > lis[7]) , 'Volume Ration'] = 0
        except:
            pass
        try:
            data.loc[(data['mrkt cap'] < lis[8]) , 'market cap'] = 1
        except:
            pass

        df = data[['sma20 > sma50','sma20 bound check', 'momentum', '8% Daily', '2% UP', 'RSI (60-72)', 'Volume Ration', \
                   'market cap', 'MACD', 'win', 'upwin', 'Loss', 'update loss']] #Selecting relevant columns

        new_line = pd.DataFrame({'Lower % bound for sma20': lis[0], \
                'Upper % bound for sma20': lis[1], 'Momentum threshold': lis[2], '% up for Day': lis[3], '% up from open': lis[4], \
                'Lower bound for RSI': lis[5], 'Upper bound for RSI': lis[6], 'Total/Average volume ratio': lis[7], \
              'Market cap threshold': lis[8] }, index=[0]) #Create new line with the combination of variable for the current iterations

        #Get the col names of 'OFF' in the list created earlier
        for c in off_column:
            col = df.columns[c]
            col_name.append(col)

        temp = df.drop(columns = col_name) #Drop all the OFF columns

        col_name.clear() #Clear the list 

        for e in event: #Loop to get the number of all events
            df_event = temp[temp[e] == 1] #Filter the dataset where event is 1
            df_event["count"] = df_event.sum(axis=1) #Create a new column and sum each row
            event_count = df_event["count"].value_counts() #Get the frequency of result at store in the series
            event_filter = event_count.to_frame() #Convert the series to Dataframe
            event_filter = event_filter.reset_index() #Reset index of the dataframe to access both sum number and its frequency
            event_filter = event_filter[event_filter['index'] == 1] #Filter the datframe to get frequency of 1 (event)
            event_filter = event_filter.reset_index(drop = True) #Reset the index so that index of filter value is 0

            #If after filter the dataframe is empty it means no event happened in this iteration give me 0 else give me count of win\
            #for that iteration and store the value in variable
            if event_filter.empty == True:
                no_of_events = 0
            else:
                no_of_events = event_filter['count'].values[0]

            new_line[e] = no_of_events #Add another column and store the event value

        result = result.append(new_line) #Add the new line in the empty dataframe created earlier

        data = data.drop(['sma20 bound check', 'momentum','8% Daily', '2% UP', 'RSI (60-72)', 'Volume Ration', 'market cap'],\
                     axis = 1) #Delete the columns created in the start of loop.
result['total'] = result['win'] + result['upwin'] + result['Loss'] + result['update loss'] #Get the total number of events
result_filter = result[result['total'] >= min_event] #Filter the iterations which has less than threshold events

#Assign weight to each event
result_filter['winweight'] = result_filter['win'] * 2
result_filter['upwinweight'] = result_filter['upwin'] * 1
result_filter['lossweight'] = result_filter['Loss'] * 2.25
result_filter['uplossweight'] = result_filter['update loss'] * 1.5

#Calculate the Win/Loss Ratio
if (result_filter['lossweight'] + result_filter['uplossweight'] == 0).any():
    result_filter['Win/Loss Ratio'] = 0
else:
    result_filter['Win/Loss Ratio'] = (result_filter['winweight'] + result_filter['upwinweight']) / (result_filter['lossweight'] + result_filter['uplossweight'])
                
result_filter = result_filter.drop(columns = ['total', 'winweight', 'upwinweight', 'lossweight', 'uplossweight'])
#Drop Unneccessary columns

result_sort = result_filter.sort_values(by = ['Win/Loss Ratio'], ascending=False) #Sort the data according Win/Loss Ratio
dynamic = result_sort.head() #Pick the top 5 iterations
dynamic.to_csv('dynamic.csv', index = False) #Save the result in CSV
dynamic #Display the result



IndexError: tuple index out of range

In [ ]:
'MACD': switch_lis[8] 'sma20 > sma50 check': switch_lis[0],